<a href="https://colab.research.google.com/github/jisoolee11/AI_study/blob/main/computer_vision(test2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> DACON 제 2회 컴퓨터 비전 학습 경진대회

In [28]:
# from google.colab import drive

# drive.mount('/content/gdrive')

In [29]:
# !unzip -qq /content/gdrive/My\ Drive/data_2/mnist_data.zip
# !unzip -qq /content/gdrive/MyDrive/data_2/test_dirty_mnist.zip
# !unzip -qq /content/gdrive/MyDrive/data_2/dirty_mnist.zip

In [30]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

from torchvision import transforms
from torchvision.models import resnet50

In [31]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(
                self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            image = self.transforms(image)

        return image, target

In [32]:
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [33]:
trainset = MnistDataset('/content', '/content/gdrive/MyDrive/data_2/dirty_mnist_answer.csv', transforms_train)
testset = MnistDataset('/content', '/content/gdrive/MyDrive/data_2/sample_submission.csv', transforms_test)

train_loader = DataLoader(trainset, batch_size=32, num_workers=2)
test_loader = DataLoader(testset, batch_size=10, num_workers=1)

In [34]:
class MnistModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet = resnet50(pretrained=True)
        self.classifier = nn.Linear(1000, 26)

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)

        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MnistModel().to(device)
print(summary(model,(3, 256, 256)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           9,408
       BatchNorm2d-2         [-1, 64, 128, 128]             128
              ReLU-3         [-1, 64, 128, 128]               0
         MaxPool2d-4           [-1, 64, 64, 64]               0
            Conv2d-5           [-1, 64, 64, 64]           4,096
       BatchNorm2d-6           [-1, 64, 64, 64]             128
              ReLU-7           [-1, 64, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          36,864
       BatchNorm2d-9           [-1, 64, 64, 64]             128
             ReLU-10           [-1, 64, 64, 64]               0
           Conv2d-11          [-1, 256, 64, 64]          16,384
      BatchNorm2d-12          [-1, 256, 64, 64]             512
           Conv2d-13          [-1, 256, 64, 64]          16,384
      BatchNorm2d-14          [-1, 256,

In [38]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MultiLabelSoftMarginLoss()

num_epochs = 10
model.train()

for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            outputs = outputs > 0.5
            acc = (outputs == targets).float().mean()
            print(f'{epoch}: {loss.item():.5f}, {acc.item():.5f}')

0: 0.04949, 0.97837
0: 0.06716, 0.96995
0: 0.06635, 0.97837
0: 0.07441, 0.97236
0: 0.07483, 0.96995
0: 0.09036, 0.96755
0: 0.10054, 0.96154
0: 0.09255, 0.96394
0: 0.08120, 0.97236
0: 0.09483, 0.96995
0: 0.07396, 0.96875
0: 0.09620, 0.96034
0: 0.07520, 0.97115
0: 0.08369, 0.97115
0: 0.06109, 0.97837
1: 0.07438, 0.96755
1: 0.08433, 0.96995
1: 0.04085, 0.98317
1: 0.10306, 0.95793
1: 0.05688, 0.97957
1: 0.06911, 0.97476
1: 0.07164, 0.97476
1: 0.10333, 0.96635
1: 0.04968, 0.97957
1: 0.05305, 0.98197
1: 0.06703, 0.97596
1: 0.07595, 0.96995
1: 0.05618, 0.97957
1: 0.06634, 0.97596
1: 0.05184, 0.97476
2: 0.08353, 0.96514
2: 0.06103, 0.97596
2: 0.06635, 0.97115
2: 0.05933, 0.97957
2: 0.06529, 0.97596
2: 0.04504, 0.98077
2: 0.07488, 0.97716
2: 0.04782, 0.98317
2: 0.06702, 0.97356
2: 0.05574, 0.98317
2: 0.04987, 0.98438
2: 0.04397, 0.97837
2: 0.04734, 0.98918
2: 0.06779, 0.97476
2: 0.05085, 0.97957
3: 0.06521, 0.97716
3: 0.04887, 0.98077
3: 0.04442, 0.98317
3: 0.03722, 0.98558
3: 0.03164, 0.98798


In [39]:
submit = pd.read_csv('/content/gdrive/MyDrive/data_2/sample_submission.csv')

model.eval()
batch_size = test_loader.batch_size
batch_index = 0
for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    outputs = outputs > 0.5
    batch_index = i * batch_size
    submit.iloc[batch_index:batch_index+batch_size, 1:] = \
        outputs.long().squeeze(0).detach().cpu().numpy()
    
submit.to_csv('/content/gdrive/MyDrive/Data/computer_vision/submit3.csv', index=False)


# from google.colab import files

# files.download('submit2.csv')